In [1]:
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
pd.set_option('display.max_rows', None)

In [3]:
url='https://immobilier.lefigaro.fr/annonces/immobilier-vente-maison-bordeaux+33200.html?sort=6&location=ile%2Bde%2Bfrance'

In [4]:
response = requests.get(url)
html_text = response.text

In [5]:
# html_text

In [6]:
soup = BeautifulSoup(response.content, 'html.parser')

In [7]:
page_list=soup.find('ol', class_='unstyled')

In [8]:
# page_list

In [9]:
store_pages=[]
for p in page_list:
    num=int(p.find('a').get_text().strip(' '))
    store_pages.append(num)
store_pages

[1, 2, 3, 100]

In [10]:
def scrap(p_min,
          p_max,
          url):
    urls_list=[]
    response_list=[]
    html_list=[]
    url=url+'&page='
    for i in range(p_min, p_max+1):
        aux_convert=str(i)
        url_=url+aux_convert
        urls_list.append(url_)
        response_ = requests.get(url_)
        time.sleep(5)
        response_list.append(response_)
        soup_ = BeautifulSoup(response_.content, 'html.parser')
        html_list.append(soup_)
    return urls_list, response_list, html_list

In [11]:
# print(soup.prettify())

In [12]:
# urls, respnses, htmls=scrap(min(store_pages), max(store_pages), url)

In [13]:
# url_df=pd.DataFrame(urls, columns=['page'])

In [14]:
# url_df['respnse']=respnses
# url_df['content']=htmls

In [15]:
# url_df.to_csv('../data/page_contents.csv', index=False)

In [16]:
url_df=pd.read_csv('../data/page_contents.csv')

In [17]:
# url_df.shape

In [18]:
url_df.head()

,page,respnse,content
0,https://immobilier.lefigaro.fr/annonces/immobi...,<Response [200]>,"<!DOCTYPE html>\n<html lang=""fr-FR""><head><tit..."
1,https://immobilier.lefigaro.fr/annonces/immobi...,<Response [200]>,"<!DOCTYPE html>\n<html lang=""fr-FR""><head><tit..."
2,https://immobilier.lefigaro.fr/annonces/immobi...,<Response [200]>,"<!DOCTYPE html>\n<html lang=""fr-FR""><head><tit..."
3,https://immobilier.lefigaro.fr/annonces/immobi...,<Response [200]>,"<!DOCTYPE html>\n<html lang=""fr-FR""><head><tit..."
4,https://immobilier.lefigaro.fr/annonces/immobi...,<Response [200]>,"<!DOCTYPE html>\n<html lang=""fr-FR""><head><tit..."


In [19]:
houses_content=[]
for soup in url_df['content']:
    soup = BeautifulSoup(soup, 'html.parser')
    house_list=soup.find_all('div', class_='cartouche-liste')
    houses_content.append(house_list)

In [20]:
url_df['houses_content']=houses_content

In [21]:
descriptions=[]
houses_descriptions=[]
for h_description in url_df['houses_content']:
    h_descp=[]
    for h in h_description:
        descp= h.get_text()
        h_descp.append(descp)
        descriptions.append(descp)
    houses_descriptions.append(h_descp)
    
url_df['houses_descriptions']=houses_descriptions

In [22]:
len(descriptions)

3100

In [23]:
type_list=[]
for soup in url_df['content']:
    soup = BeautifulSoup(soup, 'html.parser')
    house_type_list=soup.find_all('h2', class_='cartouche-liste__title')
    type_list.append(house_type_list)

In [24]:
types=[]
towns=[]
departements=[]
options=[]
houses_types=[]
for h_type in type_list:
    h_dType=[]
    for h in h_type:
        hType= h.get_text().split(' ', 2)
        h_dType.append(hType[1])
        types.append(hType[1])
        comm=hType[2].split(' (')
        towns.append(comm[0])
        dep=comm[1].split(')')
        departements.append(dep[0])
        options.append(dep[1].strip(' ').split(' '))
    houses_types.append(h_dType)  
url_df['houses_types']=houses_types

In [25]:
# departement

In [26]:
house_id=[]
for n in range(0, len(types)):
    house_id.append('H'+str(n))

In [27]:
npieces=[]
nrooms=[]
surface=[]
for o in options:
#     print(o, len(o))
    if len(o)==8:
        npieces.append(o[0])
        nrooms.append(o[3])
        surface.append(o[6])
    elif len(o)==5:
        if 'pièce' in o or 'pièces' in o and 'm²' in o:
            npieces.append(o[0])
            nrooms.append('0')
            surface.append(o[3])
        else:
            npieces.append('0')
            nrooms.append(o[0])
            surface.append(o[3])
    elif len(o)==2:
        npieces.append('0')
        nrooms.append('0')
        surface.append(o[0])
    else:
        npieces.append('0')
        nrooms.append('0')
        surface.append('0')   

In [28]:
price_list=[]
for soup in url_df['content']:
    soup = BeautifulSoup(soup, 'html.parser')
    house_price_list=soup.find_all('span', class_='price')
    price_list.append(house_price_list)
len(price_list)
url_df['prices_lists']=price_list

In [29]:
house_prices=[]
for h_prices in url_df['prices_lists']:
    for h in h_prices:
        prices= h.get_text()
        house_prices.append(prices)

In [30]:
len(house_id), len(types), len(towns), len(departements), len(options), len(npieces), len(nrooms), len(surface), len(house_prices)

(3100, 3100, 3100, 3100, 3100, 3100, 3100, 3100, 3100)

In [31]:
houses_df=pd.DataFrame({#'id':house_id,
                        'type':types,
                        'town':towns,
                        'departement':departements,
                        'pieces':npieces,
                        'rooms':nrooms,
                        'surface': surface,
                        'price':house_prices})

In [32]:
houses_df

,type,town,departement,pieces,rooms,surface,price
0,maison,Pontault-Combault,77,6,3,85,Prix NC
1,maison,L'Isle-Adam,95,8,5,277,Prix NC
2,maison,Colombes,92,7,4,115,Prix NC
3,maison,L'Isle-Adam,95,8,5,277,Prix NC
4,maison,Saint-Maurice-Montcouronne,91,6,4,135,Prix NC
5,chalet,Condé-Sainte-Libiaire,77,2,1,30,14 000 €
6,chalet,Condé-Sainte-Libiaire,77,2,1,30,24 300 €
7,ferme,La Ferté-sous-Jouarre,77,1,1,21,25 000 €
8,maison,Ocquerre,77,2,0,57,29 000 €
9,maison,Saâcy-sur-Marne,77,1,0,50,32 000 €


In [33]:
houses_df.to_csv('../data/houses_df1.csv', index=False)